In [ ]:
import json
import numpy as np


def write_json(data, fp):
    with open(fp, "w") as outfile:
        json.dump(data, outfile, indent=4)

def get_average_state_from_percent(data, percent=0.0, end_percent=1.0, normalize=True):
    """Get the average stat from a data sequence starting at a given percent.
    Parameters
    ----------
    data:  sequence
        the provided data.
    percent: float, list, tuple
        the provided percentage. Default: 0.0.
    end_percent: float, list, tuple
        the provided end percentage. Default: 1.0.
    normalize: boolean
        whether to normalize
    Returns
    -------
    result: float, list
        the computed average stat.
    """
    assert not (percent is None and end_percent is None)
    if percent is None:
        percent = end_percent
    elif end_percent is None:
        end_percent = percent
    assert isinstance(percent, (int, float, list, tuple))
    assert isinstance(end_percent, (int, float, list, tuple))
    extract_flag = False
    if isinstance(percent, (int, float)) and isinstance(end_percent, (int, float)):
        percent = [percent]
        end_percent = [end_percent]
        extract_flag = True
    elif isinstance(percent, (int, float)):
        percent = [percent] * len(end_percent)
    elif isinstance(end_percent, (int, float)):
        end_percent = [end_percent] * len(percent)
    assert len(percent) == len(end_percent)
    length = len(data)
    result = []
    for b, e in zip(percent, end_percent):
        assert b >= 0.0 and b <= 1.0
        assert e >= 0.0 and e <= 1.0
        assert e >= b
        i = int(length * b)
        j = int(length * e)
        j = j + 1 if j == i else j
        j = min(length, j)
        i = i - 1 if i == length else i
        n = j - i
        r = sum(data[i:j])
        r = r / max(1, n) if normalize else r
        result.append(r)
    if len(result) == 1 and extract_flag:
        result = result[0]
    return result




提供的代码片段包括两个实用函数： write_json(data, fp)：此函数将给定数据写入位于指定文件路径 (fp) 的 JSON 文件。数据以 4 个空格的缩进级别写入。 get_average_state_from_percent(data, percent=0.0, end_percent=1.0, normalize=True)：此函数计算从给定百分比开始到结束百分比结束的数据序列的平均统计数据。参数如下： 数据：数据的输入序列。 percent：数据序列的起始百分比。它可以是单个值或值的列表/元组。默认值为 0.0。 end_percent：数据序列的结束百分比。它可以是单个值或值的列表/元组。默认值为 1.0。 normalize：一个布尔值，指示是否对计算的平均值进行归一化。如果为 True，则平均值除以所选数据范围内的元素数。默认为真。 该函数处理 percent 和 end_percent 参数的不同情况。它支持单个值或具有不同组合的多个值。它计算每个指定范围的平均统计数据，并根据输入将结果作为单个值或列表返回。 请注意，该函数假定数据是一个序列（例如，列表、numpy 数组）并执行切片操作以提取所需的范围。

In [ ]:
def compute_severity_stats(preds, gt_diff, severity_mask, diff_proba_threshold):
    if severity_mask is None:
        severity_mask = [0] * preds.shape[-1]
    if len(preds.shape) > len(gt_diff.shape):
        tmp_shape = ([1] * (len(preds.shape) - len(gt_diff.shape))) + list(gt_diff.shape)
        gt_diff = gt_diff.reshape(tmp_shape)
    sev_shape = ([1] * (len(gt_diff.shape) - 1)) + [-1]
    severity_mask= np.array(severity_mask).reshape(sev_shape)
    gt_severity_mask = ((gt_diff > diff_proba_threshold) * severity_mask).astype(bool)
    pr_severity_mask = ((preds > diff_proba_threshold) * severity_mask).astype(bool)
    # mask of severe patho in pred not in gt
    pr_gt_nonshared_severity_mask = (np.logical_not(gt_severity_mask) * pr_severity_mask).astype(bool)
    # mask of severe patho in gt not in pred
    gt_pr_nonshared_severity_mask = (np.logical_not(pr_severity_mask) * gt_severity_mask).astype(bool)
    nb_out = pr_gt_nonshared_severity_mask.sum(-1)
    nb_in = gt_pr_nonshared_severity_mask.sum(-1)
    gt_nbSev = gt_severity_mask.astype(int).sum(-1)
    max_sev = severity_mask.sum()
    pred_no_gt = (max_sev - gt_nbSev - nb_out) / np.maximum(1, max_sev - gt_nbSev)
    gt_no_pred = (gt_nbSev - nb_in) / np.maximum(1, gt_nbSev)
    gt_pred_f1 = compute_f1(pred_no_gt, gt_no_pred)
    return pred_no_gt, gt_no_pred, gt_pred_f1
    
    

def kl_trajectory_auc(kl_explore, kl_confirm):
    kl_explore = np.array(kl_explore)
    kl_confirm = np.array(kl_confirm)
    result = np.trapz(kl_explore, kl_confirm, axis=-1)
    return result

def kl_confirm_score(probas, dist, c=1):
    entropy = -np.sum(dist * np.log(dist + 1e-10), axis=-1)
    if len(probas.shape) > len(dist.shape):
        tmp_shape = ([1] * (len(probas.shape) - len(dist.shape))) + list(dist.shape)
        dist = dist.reshape(tmp_shape)
    cross_entropy = -np.sum(dist * np.log(probas + 1e-10), axis=-1)
    kl_val = np.maximum(0, cross_entropy - entropy)
    kl_val = np.exp(-c * kl_val)
    return kl_val




提供的代码片段包括三个函数： compute_severity_stats(preds, gt_diff, severity_mask, diff_proba_threshold)：此函数根据预测 (preds)、地面实况差异 (gt_diff)、严重性掩码 (severity_mask) 和微分概率阈值 (diff_proba_threshold) 计算严重性统计数据。参数如下： preds：预测的差异。 gt_diff：地面实况差异。 severity_mask：表示每种病理学严重程度的掩码。它用于根据病症的严重程度过滤掉病症。如果没有，则使用全零的默认严重性掩码。 diff_proba_threshold：用于确定病理是否严重的微分概率阈值。 该函数执行各种计算以计算不同的严重性统计数据，包括预测中不存在于基本事实中的严重病理学数量、基本事实中不存在于预测中的严重病理学数量以及 F1 分数与严重病理学预测但不存在于基本事实中，以及严重病理学在基本事实中但不存在于预测中的数量。该函数将计算的统计信息返回为 pred_no_gt、gt_no_pred 和 gt_pred_f1。 kl_trajectory_auc(kl_explore, kl_confirm)：此函数计算由 kl_explore 和 kl_confirm 定义的轨迹的曲线下面积 (AUC)。参数如下： kl_explore：沿轨迹的探索值（例如，KL 散度）。 kl_confirm：沿轨迹的确认值（例如，KL 散度）。 该函数使用梯形规则来近似曲线下的面积并将结果作为结果返回。 kl_confirm_score(probas, dist, c=1)：此函数根据预测概率 (probas) 和真实分布 (dist) 计算确认分数。参数 c 是一个比例因子，用于控制确认分数的影响。该函数计算真实分布的熵、预测概率与真实分布之间的交叉熵，然后将确认分数计算为交叉熵与熵之差的指数乘以比例因子 c。该函数将确认分数作为 kl_val 返回。

In [ ]:
def kl_explore_score(probas, first_proba=None, c=1):
    dist = probas[0] if first_proba is None else first_proba
    entropy = -np.sum(dist * np.log(dist + 1e-10), axis=-1)
    if len(probas.shape) > len(dist.shape):
        tmp_shape = ([1] * (len(probas.shape) - len(dist.shape))) + list(dist.shape)
        dist = dist.reshape(tmp_shape)
    cross_entropy = -np.sum(dist * np.log(probas + 1e-10), axis=-1)
    kl_val = np.maximum(0, cross_entropy - entropy)
    kl_val = np.exp(-c * kl_val)
    kl_val = 1.0 - kl_val
    return kl_val
    
def kl_trajectory_score(kl_explore, kl_confirm, alphas=None):
    kl_explore = np.array(kl_explore)
    kl_confirm = np.array(kl_confirm)
    if alphas is None:
        if kl_explore.shape[-1] > 1:
            alphas = np.arange(start=kl_explore.shape[-1] - 1, stop=-1, step=-1)
            alphas = alphas / (kl_explore.shape[-1] - 1)
            if len(kl_explore.shape) > len(alphas.shape):
                tmp_shape = ([1] * (len(kl_explore.shape) - len(alphas.shape))) + list(alphas.shape)
                alphas = alphas.reshape(tmp_shape)
        else:
            alphas = 0.5
    score = alphas * kl_explore + (1 - alphas) * kl_confirm
    return score

def compute_f1(p, r):
    if isinstance(p, (list, tuple)):
        p = np.array(p)
    if isinstance(r, (list, tuple)):
        r = np.array(r)
    denom = p + r
    return (2 * p * r) / (denom + 1e-10)
    



提供的代码片段包括三个附加功能： kl_explore_score(probas, first_proba=None, c=1)：此函数根据预测概率 (probas) 计算探索分数。参数 first_proba 是可选参数，表示初始概率分布。参数 c 是一个比例因子，用于控制探索分数的影响。该函数计算初始分布（或第一概率分布）的熵，预测概率与初始分布之间的交叉熵，然后计算探索得分作为交叉熵与熵之间差异的指数，乘以比例因子 c。该函数将探索分数作为 kl_val 返回。通过从 1 中减去 exploration score 得到最终分数，进一步转换 exploration score。 kl_trajectory_score(kl_explore, kl_confirm, alphas=None)：此函数根据探索分数 (kl_explore) 和确认分数 (kl_confirm) 计算轨迹分数。参数 alphas 是一个可选参数，用于指定在计算轨迹分数时分配给探索分数和确认分数的权重。如果未提供 alphas，则使用默认的线性加权方案。该函数使用指定的权重计算探索分数和确认分数的加权和，并将轨迹分数作为分数返回。 compute_f1(p, r)：此函数根据精度 (p) 和召回率 (r) 计算 F1 分数。精度和召回值可以列表、元组或 numpy 数组的形式提供。该函数使用公式 (2 * p * r) / (p + r + 1e-10) 计算 F1 分数并返回结果。

In [ ]:
def compute_metrics(gt_differential, disease, final_diags, all_diags, valid_timesteps, present_evidences, inquired_evidences, symptom_mask, atcd_mask, severity_mask, tres=0.01):
        all_indices = list(range(disease.shape[0]))
        top_ranked = np.argsort(final_diags, axis=-1)
        top_ranked = top_ranked[:, ::-1]
        gt_diff_top_ranked = np.argsort(gt_differential, axis=-1)
        gt_diff_top_ranked = gt_diff_top_ranked[:, ::-1]
        all_len = np.sum(valid_timesteps, axis=-1) + 1
        
        result = {}
        result["IL"] = np.mean(all_len)
        result["GTPA"] = np.mean(final_diags[all_indices, disease] > tres)
        result["GTPA@1"] = np.mean(np.logical_and(disease == top_ranked[:, 0], final_diags[all_indices, disease] > tres))
        result["GTPA@3"] = np.mean(np.logical_and(np.any(disease.reshape(-1, 1) == top_ranked[:, 0:3], axis=-1), final_diags[all_indices, disease] > tres))
        result["GTPA@5"] = np.mean(np.logical_and(np.any(disease.reshape(-1, 1) == top_ranked[:, 0:5], axis=-1), final_diags[all_indices, disease] > tres))

        gt_diff_mask = (gt_differential > tres)
        pred_diff_mask = (final_diags > tres)

        ddr = np.sum(np.logical_and(gt_diff_mask, pred_diff_mask), axis=-1) / np.maximum(1, np.sum(gt_diff_mask, axis=-1))
        ddp = np.sum(np.logical_and(gt_diff_mask, pred_diff_mask), axis=-1) / np.maximum(1, np.sum(pred_diff_mask, axis=-1))
        ddf1 = compute_f1(ddp, ddr)

        result[f"DDR"] = np.mean(ddr)
        result[f"DDP"] = np.mean(ddp)
        result[f"DDF1"] = np.mean(ddf1)
        for k in [1, 3, 5]:
            tmp_gt_k = np.zeros_like(gt_diff_mask).astype(bool)
            np.put_along_axis(tmp_gt_k, gt_diff_top_ranked[:, 0:k], True, 1)
            tmp_gt_k = np.logical_and(gt_diff_mask, tmp_gt_k)

            tmp_pred_k = np.zeros_like(pred_diff_mask).astype(bool)
            np.put_along_axis(tmp_pred_k, top_ranked[:, 0:k], True, 1)
            tmp_pred_k = np.logical_and(pred_diff_mask, tmp_pred_k)

            tmp_ddr = np.sum(np.logical_and(tmp_gt_k, tmp_pred_k), axis=-1) / np.maximum(1, np.sum(tmp_gt_k, axis=-1))
            tmp_ddp = np.sum(np.logical_and(tmp_gt_k, tmp_pred_k), axis=-1) / np.maximum(1, np.sum(tmp_pred_k, axis=-1))
            tmp_ddf1 = compute_f1(tmp_ddp, tmp_ddr)
            result[f"DDR@{k}"] = np.mean(tmp_ddr)
            result[f"DDP@{k}"] = np.mean(tmp_ddp)
            result[f"DDF1@{k}"] = np.mean(tmp_ddf1)

        dsp, dsr, dsf1 = compute_severity_stats(final_diags, gt_differential, severity_mask, tres)
        result["DSP"] = np.mean(dsp)
        result["DSR"] = np.mean(dsr)
        result["DSF1"] = np.mean(dsf1)

        pos_evi = np.logical_and(present_evidences, inquired_evidences)
        per = np.sum(pos_evi, axis=-1) / np.maximum(1, np.sum(present_evidences, axis=-1))
        pep = np.sum(pos_evi, axis=-1) / np.maximum(1, np.sum(inquired_evidences, axis=-1))
        pef1 = compute_f1(pep, per)

        result["PER"] = np.mean(per)
        result["PEP"] = np.mean(pep)
        result["PEF1"] = np.mean(pef1)

        present_symptoms = np.logical_and(present_evidences, symptom_mask.reshape(1, -1))
        inquired_symptoms = np.logical_and(inquired_evidences, symptom_mask.reshape(1, -1))
        pos_symp = np.logical_and(present_symptoms, inquired_symptoms)
        psr = np.sum(pos_symp, axis=-1) / np.maximum(1, np.sum(present_symptoms, axis=-1))
        psp = np.sum(pos_symp, axis=-1) / np.maximum(1, np.sum(inquired_symptoms, axis=-1))
        psf1 = compute_f1(psp, psr)

        result["PSR"] = np.mean(psr)
        result["PSP"] = np.mean(psp)
        result["PSF1"] = np.mean(psf1)

        present_atcds = np.logical_and(present_evidences, atcd_mask.reshape(1, -1))
        inquired_atcds = np.logical_and(inquired_evidences, atcd_mask.reshape(1, -1))
        pos_atcd = np.logical_and(present_atcds, inquired_atcds)
        par = np.sum(pos_atcd, axis=-1) / np.maximum(1, np.sum(present_atcds, axis=-1))
        pap = np.sum(pos_atcd, axis=-1) / np.maximum(1, np.sum(inquired_atcds, axis=-1))
        paf1 = compute_f1(pap, par)

        result["PAR"] = np.mean(par)
        result["PAP"] = np.mean(pap)
        result["PAF1"] = np.mean(paf1)


        tmp_shape = list(gt_differential.shape)
        tmp_shape = tmp_shape[0:1] + [1] + tmp_shape[1:]
        gt_diff_proba = gt_differential.reshape(tmp_shape)
        confirm_score = kl_confirm_score(all_diags, gt_diff_proba)
        explore_score = kl_explore_score(all_diags, first_proba=all_diags[:, 0:1])
        succesive_explore_score = kl_explore_score(all_diags[:, 1:], first_proba=all_diags[:, 0:-1])

        pred_no_gt, gt_no_pred, gt_pred_f1 = compute_severity_stats(all_diags, gt_diff_proba, severity_mask, tres)

        p = list(range(0, 105, 5))
        # p_idx = {v: i for i, v in enumerate(p)}
        p = [v / 100.0 for v in p]
        
        t_explore_score = 0
        t_succesive_explore_score = 0
        t_confirm_score = 0
        t_kl_trajectory_values = 0
        t_pred_no_gt = 0
        t_gt_no_pred = 0
        t_gt_pred_f1 = 0
        kl_trajectory_values_sum = 0
        kl_trajectory_auc_sum = 0
        tvd_sum = 0
        for i in range(len(all_len)):
            if all_len[i] == 0:
                continue
            mini_prob = np.amin(all_diags[i, 0:all_len[i]], axis=0)
            maxi_prob = np.amax(all_diags[i, 0:all_len[i]], axis=0)    
            tvd_sum += np.absolute(maxi_prob - mini_prob).mean()
            kl_trajectory_values = kl_trajectory_score(explore_score[i, 0:all_len[i]], confirm_score[i, 0:all_len[i]])
            kl_trajectory_values_sum += np.mean(kl_trajectory_values)
            kl_trajectory_auc_sum += kl_trajectory_auc(explore_score[i, 0:all_len[i]], confirm_score[i, 0:all_len[i]])
            t_explore_score = t_explore_score + np.array(get_average_state_from_percent(explore_score[i, 0:all_len[i]], percent=p, end_percent=None))
            t_succesive_explore_score = t_succesive_explore_score + np.array(get_average_state_from_percent(succesive_explore_score[i, 0:all_len[i]-1], percent=p, end_percent=None))
            t_confirm_score = t_confirm_score + np.array(get_average_state_from_percent(confirm_score[i, 0:all_len[i]], percent=p, end_percent=None))
            t_kl_trajectory_values = t_kl_trajectory_values + np.array(get_average_state_from_percent(kl_trajectory_values, percent=p, end_percent=None))
            t_pred_no_gt = t_pred_no_gt + np.array(get_average_state_from_percent(pred_no_gt[i, 0:all_len[i]], percent=p, end_percent=None))
            t_gt_no_pred = t_gt_no_pred + np.array(get_average_state_from_percent(gt_no_pred[i, 0:all_len[i]], percent=p, end_percent=None))
            t_gt_pred_f1 = t_gt_pred_f1 + np.array(get_average_state_from_percent(gt_pred_f1[i, 0:all_len[i]], percent=p, end_percent=None))
        
        tvd_sum /= max(1, len(all_len))
        kl_trajectory_values_sum /= max(1, len(all_len))
        kl_trajectory_auc_sum /= max(1, len(all_len))
        t_explore_score /= max(1, len(all_len))
        t_succesive_explore_score /= max(1, len(all_len))
        t_confirm_score /= max(1, len(all_len))
        t_kl_trajectory_values /= max(1, len(all_len))
        t_pred_no_gt /= max(1, len(all_len))
        t_gt_no_pred /= max(1, len(all_len))
        t_gt_pred_f1 /= max(1, len(all_len))

        result["TVD"] = tvd_sum
        result["TrajScore"] = kl_trajectory_values_sum
        result["AUCTraj"] = kl_trajectory_auc_sum
        result["PlotData.x"] = np.array(p)
        result["PlotData.Exploration"] = t_explore_score
        result["PlotData.SuccessiveExploration"] = t_succesive_explore_score
        result["PlotData.Confirmation"] = t_confirm_score
        result["PlotData.Trajectory"] = t_kl_trajectory_values
        result["PlotData.SevF1"] = t_gt_pred_f1
        result["PlotData.SevPrecOut"] = t_pred_no_gt
        result["PlotData.SevRecIn"] = t_gt_no_pred
        
        return result

compute_metrics 函数接受多个输入并根据这些输入计算各种评估指标。以下是该函数作用的概述： 该函数初始化一个名为 result 的空字典来存储计算的指标。 该函数计算有效时间步长的平均长度 (all_len) 并将其分配给结果 ["IL"]。 该函数计算不同截止点 (tres) 的地面真实准确度 (GTPA) 指标，并将它们分配给 result["GTPA"]、result["GTPA@1"]、result["GTPA@3"] 和 result[ “GTPA@5”]。 该函数计算不同截止值 (tres) 的鉴别诊断召回率 (DDR)、鉴别诊断精度 (DDP) 和鉴别诊断 F1 分数 (DDF1) 指标，并将它们分配给 result["DDR"], result["DDP" ]，结果[“DDF1”]。 该函数计算不同截止点 (k) 的 DDR、DDP 和 DDF1 指标，并将它们分配给 result["DDR@k"]、result["DDP@k"] 和 result["DDF1@k"] k值。 该函数根据预测疾病的严重程度计算疾病严重程度精度 (DSP)、疾病严重程度召回 (DSR) 和疾病严重程度 F1 分数 (DSF1) 指标，并将它们分配给 result["DSP"], result[" DSR"] 和结果 ["DSF1"]。 该函数根据证据的存在计算当前证据率 (PER)、当前证据精度 (PEP) 和当前证据 F1 分数 (PEF1) 指标，并将它们分配给 result["PER"], result["PEP" ]，结果[“PEF1”]。 该函数根据症状的存在计算出现症状率 (PSR)、出现症状精度 (PSP) 和出现症状 F1 分数 (PSF1) 指标，并将它们分配给 result["PSR"], result["PSP" ]，结果[“PSF1”]。 该函数根据存在的病史 (ATCD) 计算当前 ATCD 率 (PAR)、当前 ATCD 精度 (PAP) 和当前 ATCD F1 分数 (PAF1) 指标，并将它们分配给结果 ["PAR"]，结果[“PAP”]，和结果[“PAF1”]。 该函数重塑预测的鉴别诊断概率，并根据这些概率计算确认分数、探索分数和连续探索分数。 该函数计算不同截止值 (tres) 的预测无地面真实情况、地面真实情况无预测和地面真实情况预测 F1 分数指标，并将它们分配给 t_pred_no_gt、t_gt_no_pred 和 t_gt_pred_f1。 该函数计算最小和最大概率之间的总变异距离 (TVD)，并将其分配给 result["TVD"]。 该函数根据确认和探索分数计算轨迹分数和轨迹 AUC（曲线下面积），并将它们分配给 result["TrajScore"] 和 result["AUCTraj"]。 该函数创建一个百分比值 (p) 数组，并使用 get_average_state_from_percent 函数计算每个指标在不同百分比截止点的平均状态。计算出的平均状态被分配给结果字典中的相应键。 最后，该函数返回包含所有计算指标的结果字典。 请注意，一些函数如 compute_f1、kl_confirm_score、kl_explore_score、kl_trajectory_score、kl_trajectory_auc 和 get_average_state_from_percent 假定在代码的其他地方定义。